TUGAS MEMBUAT DETEKSI MATA TERBUKA TERTUTUP DAN MULUT TERBUKA

In [1]:
import cv2
import mediapipe as mp
import math
import logging

In [3]:
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Fungsi untuk hitung jarak
def distance(p1, p2):
    return ((p1.x - p2.x)**2 + (p1.y - p2.y)**2) ** 0.5

# Inisialisasi Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

# Buka Webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    logging.error("Gagal membuka webcam. Pastikan kamera tersedia dan tidak digunakan aplikasi lain.")
    exit()

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            logging.warning("Gagal membaca frame dari webcam.")
            break

        # Preprocess
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)

        labels = []

        if results.multi_face_landmarks:
            for landmarks in results.multi_face_landmarks:
                # Ambil titik-titik untuk mata kiri
                left_eye_top = landmarks.landmark[159]
                left_eye_bottom = landmarks.landmark[145]
                # Ambil titik-titik untuk mata kanan
                right_eye_top = landmarks.landmark[386]
                right_eye_bottom = landmarks.landmark[374]
                # Ambil titik-titik bibir
                top_lip = landmarks.landmark[13]
                bottom_lip = landmarks.landmark[14]

                # Jarak vertikal mata
                left_eye_distance = distance(left_eye_top, left_eye_bottom)
                right_eye_distance = distance(right_eye_top, right_eye_bottom)
                # Jarak vertikal bibir
                mouth_open = distance(top_lip, bottom_lip)

                # Threshold sederhana (bisa di-tune)
                if left_eye_distance < 0.02 and right_eye_distance < 0.02:
                    labels.append("Mata Tertutup")
                else:
                    labels.append("Mata Terbuka")
                
                if mouth_open > 0.05:
                    labels.append("Mulut Terbuka")
                else:
                    labels.append("Mulut Tertutup")

        # Tampilkan label prediksi
        label_text = " | ".join(labels) if labels else "Tidak Terdeteksi"
        cv2.putText(frame, f'Prediksi: {label_text}', (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Tampilkan frame
        try:
            cv2.imshow("Deteksi Ekspresi Real-time", frame)
        except cv2.error as e:
            logging.error(f"Gagal menampilkan window: {e}")
            break

        # Tombol keluar
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            logging.info("Program dihentikan oleh pengguna melalui tombol 'q'.")
            break

except KeyboardInterrupt:
    logging.info("Program dihentikan secara paksa oleh pengguna (Ctrl+C).")
except Exception as e:
    logging.error(f"Terjadi kesalahan: {e}")

finally:
    # Bersihkan sumber daya
    cap.release()
    cv2.destroyAllWindows()
    face_mesh.close()  # Tutup FaceMesh untuk membebaskan sumber daya
    logging.info("Sumber daya webcam, window, dan FaceMesh dibersihkan.")

2025-04-21 00:36:58,001 - INFO - Program dihentikan secara paksa oleh pengguna (Ctrl+C).
2025-04-21 00:36:58,491 - INFO - Sumber daya webcam, window, dan FaceMesh dibersihkan.
